![holoviz logo](https://holoviz.org/assets/holoviz-logo-stacked.svg)

In this tutorial, we will look into using [HoloViz](https://holoviz.org/index.html) in the specific context of using it in combination with Vega-Lite and Vega for data visualisation from python.

HoloViz brings together a group of tools to simplifies visualisation in python. These tools are:

- [Panel](https://panel.holoviz.org/), which we’ll use for building dashboards
- [HoloViews](https://holoviews.org/), which makes it possible to very quickly create data visualisation
- [hvPlot](https://hvplot.holoviz.org/) is a high-level plotting API built on HoloViews
- [GeoViews](http://geoviews.org/), which extends HoloViews for geographic data
- [Datashader](https://datashader.org/), which allows rendering huge datasets
- [Param](https://param.holoviz.org/), which helps to create user-configurable objects
- [Colorcet](https://colorcet.holoviz.org/), which defines perceptually uniform colormaps.

We’ll be using Panel and Param in this tutorial, within the context of [jupyter notebooks](https://jupyter.org/).

Much of this tutorial is based on the excellent tutorial given by James Bednar available at https://www.youtube.com/watch?v=7deGS4IPAQ0.

# A minimal vega plot

As a proof-of-principle, let’s recreate the very first plot we made in the VegaLite tutorial: the simple barchart.

## Loading the necessary libraries
First things first: loading the necessary libraries. Make sure that these are installed first (see the previous step).

In [ ]:
from vega import Vega

At this point we won’t need `panel` and/or `param` yet.

## Creating the specification
We’ll just put the whole specification in a variable that we call spec.

In [ ]:
spec = {
    "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
    "description": "A simple bar chart with embedded data.",
    "data": {
      "values": [
        {"a": "A", "b": 28},
        {"a": "B", "b": 55},
        {"a": "C", "b": 43},
        {"a": "D", "b": 91}
      ]
    },
    "mark": "bar",
    "encoding": {
      "x": {"field": "b", "type": "quantitative"},
      "y": {"field": "a", "type": "nominal"}
    }
  }


## Drawing the visualisation
And finally we call the `Vega()` function, passing it the spec as an argument. This will give us the actual plot.

In [ ]:
Vega(spec)

# Splitting the specification

In the previous step, we created a variable spec that contained the complete specification for the plot. One of the reasons we’re looking into using jupyter notebooks instead of the online vega editor, is that we can make a very long specification more manageable by splitting it up: we can extract different parts of the spec into separate variables.

## A simple example
Let’s extract the dataset itself into a separate variable:

In [ ]:
d = { "values":
          [ {"a": "A", "b": 28},
            {"a": "B", "b": 55},
            {"a": "C", "b": 43},
            {"a": "D", "b": 91} ]
        }

… and replace the dataset in the specification with that variable:

In [ ]:
spec = {
    "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
    "description": "A simple bar chart with embedded data.",
    "data": d,
    "mark": "bar",
    "encoding": {
      "x": {"field": "b", "type": "quantitative"},
      "y": {"field": "a", "type": "nominal"}
    }
  }

This will have the same result as we had earlier and means that we can build up our dataset within the jupyter notebook and then pass it on to the specification.

## A more extensive example
Here’s a longer example (from /visualisation-tutorial/vegalite-brushing-and-linking.html) that makes it even more clear that it might be good to create the different sections of the specification separately.

The data:

In [ ]:
cars_dataset = {
    "name": "cars",
    "url": "https://raw.githubusercontent.com/vega/vega/master/docs/data/cars.json",
    "format": {"type": "json"}
  }

The scales:

In [ ]:
scales = [
  {
    "name": "acceleration_xscale",
    "type": "linear",
    "domain": {"data": "cars", "field": "Acceleration"},
    "range": [0, 200],
    "nice": true,
    "zero": true
  },
  {
    "name": "mpg_yscale",
    "type": "linear",
    "domain": {"data": "cars", "field": "Miles_per_Gallon"},
    "range": [200, 0],
    "nice": true,
    "zero": true
  },
  {
    "name": "horsepower_xscale",
    "type": "linear",
    "domain": {"data": "cars", "field": "Horsepower"},
    "range": [0, 200],
    "nice": true,
    "zero": true
  }
]

This will give you an error, because the true used in the Vega specification is lowercase, while a True in python needs to be capitalised. So we have to change this first:

In [ ]:
scales = [
  {
    "name": "acceleration_xscale",
    "type": "linear",
    "domain": {"data": "cars", "field": "Acceleration"},
    "range": [0, 200],
    "nice": True,
    "zero": True
  },
  {
    "name": "mpg_yscale",
    "type": "linear",
    "domain": {"data": "cars", "field": "Miles_per_Gallon"},
    "range": [200, 0],
    "nice": True,
    "zero": True
  },
  {
    "name": "horsepower_xscale",
    "type": "linear",
    "domain": {"data": "cars", "field": "Horsepower"},
    "range": [0, 200],
    "nice": True,
    "zero": True
  }
]

In all the code below, I’ve already made this change.

The axes:

In [ ]:
mpg_y_axis = {
  "scale": "mpg_yscale",
  "orient": "left",
  "gridScale": "mpg_yscale",
  "grid": True,
  "tickCount": 5,
  "title": "Miles_per_Gallon"
}

acceleration_x_axis = {
  "scale": "acceleration_xscale",
  "orient": "bottom",
  "gridScale": "acceleration_xscale",
  "grid": True,
  "tickCount": 5,
  "title": "Acceleration"
}

horsepower_x_axis = {
  "scale": "horsepower_xscale",
  "orient": "bottom",
  "gridScale": "horsepower_xscale",
  "grid": True,
  "tickCount": 5,
  "title": "Horse power"
}

The plots themselves:

In [ ]:
plot1 = {
  "type": "group",
  "encode": {
    "update": {
      "width": {"value": 200},
      "height": {"value": 200}
    }
  },
  "marks": [
    {
      "type": "symbol",
      "style": "circle",
      "from": {"data": "cars"},
      "encode": {
        "update": {
          "x": {"scale": "acceleration_xscale", "field": "Acceleration"},
          "y": {"scale": "mpg_yscale", "field": "Miles_per_Gallon"},
          "fill": {"value": "steelblue"},
          "fillOpacity": {"value": 0.5}
        }
      }
    }
  ],
  "axes": [ mpg_y_axis, acceleration_x_axis ]
}

plot2 = {
  "type": "group",
  "style": "cell",
  "encode": {
    "update": {
      "width":  {"value": 200},
      "height": {"value": 200}
    }
  },
  "marks": [
    {
      "type": "symbol",
      "style": "circle",
      "from": {"data": "cars"},
      "encode": {
        "enter": {
          "x": {"scale": "horsepower_xscale", "field": "Horsepower"},
          "y": {"scale": "mpg_yscale", "field": "Miles_per_Gallon"},
          "fill": {"value": "steelblue"},
          "fillOpacity": {"value": 0.5}
        }
      }
    }
  ],
  "axes": [ mpg_y_axis, horsepower_x_axis ]
}

This makes the final specification much more manageable:

In [ ]:
spec2 = {
  "$schema": "https://vega.github.io/schema/vega/v5.json",
  "padding": 5,
  "data": [ cars_dataset ],
  "scales": scales,
  "layout": {"padding": 20},
  "marks": [
    plot1,
    plot2
  ]
}

The result is the following plot (that we also created in the vega tutorial).

In [ ]:
Vega(spec2)

We’ve seen in the debugging section of the Vega tutorial that the online vega editor has some very useful features for writing correct vega code. We don’t have those here, but being able to split up the code in smaller parts can help a lot, and we can build these parts up within the notebook.

# What is panel?

The Panel library adds functionality to jupyter notebook for quickly and easily building interactive dashboards. There are 2 main concepts:

- panel = a container of things
- pane = one of these things. This could also be a widget.

In the image below, the red rectangle is the “panel” and the green ones are “panes”.

![panel and pane](https://vda-lab.github.io/visualisation-tutorial/assets/panel-with-panes.png)

## Setting things up
You’ll have installed panel as instructed above. To load it into jupyter notebook, run

In [ ]:
import panel as pn
pn.extension()

Proof-of-principle: we can now create a simple panel like this:

In [ ]:
pn.pane.Markdown("# This is a markdown title")

This should show a title.

## What are the different types of panes?
Just using markdown does not seem that interesting, but we can put other stuff in a pane as well, like images, vega code(!), etc. To find out what you can use, type `pn.pane.` and then a tab. This will show you all possibilities: `bokeh`, `dataframe`, `gif`, `html`, `image`, `latex`, `plotly`, `svg`, `video`, `vtk`, etc.

## Panel and Google Colaboratory
Google Colaboratory is a great way to use jupyter notebooks. Access them through http://colab.research.google.com or straight from your google drive. Unfortunately, there are some quirks to take into account when working with Panel.

- You have to install panel and other modules every time again in your notebook with `!pip install panel`, `!pip install vega`, etc.
- Whereas you only have to write `pn.extension()` once in a jupyter notebook, you have to do this in every cell in Colaboratory in which you use `Panel`.
- You can’t use the `.show()` method in Colaboratory (see below).

# Panel layout

To combine different plots in vega, we needed to create a `layout` with a `marks` section which itself contained other `marks` (see [here](https://vda-lab.github.io/visualisation-tutorial/vega-composing-plots.html)). Something like this:
```json
...
"layout": {"padding": 20},
  "marks": [
    {
      "type": "group",
      "encode": {
        "update": {
          "width": {"value": 200},
          "height": {"value": 200}
        }
      },
      "marks": [
        {
          "type": "symbol",
          ...
        }
      ],
      "axes": [
        ...
      ]
    },
    {
      "type": "group",
      "style": "cell",
      "encode": {
        "update": {
          "width":  {"value": 200},
          "height": {"value": 200}
        }
      },
      "marks": [
        {
          "type": "symbol",
          ...
        }
      ],
      "axes": [
        ...
      ]
    }
  ]
}
...
```
It’s not easy to keep an overview of what’s going on here, but panel makes things much more clear. We can layout different panes into a panel with rows, columns or tabs with `pn.Column()`, `pn.Row()` and `pn.Tabs()`, respectively. For example:

In [ ]:
pn.Column(pn.pane.Markdown("# My title"),
          pn.pane.Markdown("Some text"))

We can make the layout more complex by combining different tags:

In [ ]:
pn.Column(pn.pane.Markdown("# My title"),
          pn.Row(pn.pane.Markdown("Some text"),
                 pn.pane.Markdown("Some other text")))

…and combining different types of panes:

In [ ]:
pn.Column(pn.pane.Markdown("# My title"),
          pn.Row(pn.pane.Markdown("The first hit when I search google images for the text 'panel'"),
                 pn.pane.JPG("https://5.imimg.com/data5/PA/LN/MY-13921/dol-starter-panel-500x500.jpg")))

_Exercise - Try out different layouts, including tabs_

# Panel is reactive

One of the strong points of panel is that is reactive. As we described in the vega tutorial, in a reactive program any change in a variable will automatically be reflected in other variables that depend on it.

Let’s create a small panel with a title and a bit of text:

In [ ]:
title = pn.pane.Markdown("# This is my title")
text = pn.pane.Markdown("This is the text")
pn.Column(title, text)

The `title` variable is much more than just a bit of text. You can see that if you check what methods are available on it: write `title.` (with the period) and try tab-completion. Calling `title.object` will give you the text contained in the variable: `# This is my title`.

In [ ]:
title.object

We can use that to change the text. When running the next code block, check what is happening to the text above:

In [ ]:
title.object = "# Title is changed!"

As you can see the title in the output above is automatically changed from `This is my title` to `Title is changed!`. So you don’t have to re-run cells in the notebook to update their output, and any data streams will be dynamically reflected.

# Panel and vega

So let’s finally combine Panel with Vega. As for creating a pane with markdown text or a figure, we can use `pn.pane.Vega()` and give it a specification.

In [ ]:
import panel as pn
from vega import Vega
pn.extension('vega')

spec = {
    "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
    "description": "A simple bar chart with embedded data.",
    "width": 200,
    "height": 100,
    "data": {
      "values": [
        {"a": "A", "b": 28},
        {"a": "B", "b": 55},
        {"a": "C", "b": 43},
        {"a": "D", "b": 91}
      ]
    },
    "mark": "bar",
    "encoding": {
      "x": {"field": "b", "type": "quantitative"},
      "y": {"field": "a", "type": "nominal"}
    }
  }
pn.pane.Vega(spec)

Now we can combine this with other panes. In the Vega and VegaLite tutorials, we defined the title within the specification itself. Now, instead, we can define it outside:

In [ ]:
pn.Column(pn.pane.Markdown("# This is a horizontal barchart"),
          pn.Row(
                 pn.panel("This barchart with 4 categories shows us how we can combine different panes, including vega plots"),
                 pn.pane.Vega(spec)

          ))

Or we can combine two plots:

In [ ]:
pn.Column(pn.pane.Markdown("# These are two horizontal barcharts"),
          pn.Row(
                 pn.pane.Vega(spec),
                 pn.pane.Vega(spec)

          ))

By the way, do not forget to specify width and height of the plot, because panel cannot deduce that automatically. If you forget to do so, you’ll get overlapping panels like this:

In [ ]:
spec2 = {
    "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
    "description": "A simple bar chart with embedded data.",
    "width": 200,
    "data": {
      "values": [
        {"a": "A", "b": 28},
        {"a": "B", "b": 55},
        {"a": "C", "b": 43},
        {"a": "D", "b": 91}
      ]
    },
    "mark": "bar",
    "encoding": {
      "x": {"field": "b", "type": "quantitative"},
      "y": {"field": "a", "type": "nominal"}
    }
  }
pn.pane.Vega(spec2)

# Sharing dashboards

Panel has another neat trick: it can create a dashboard that is separate from the whole jupyter notebook interface.

In the previous section, we created a dashboard with a title, some text and a plot. However, this code is embedded in the rest of the notebook. Obviously, this is good enough if we’re still in the developing phase or or exploring data yourself, but you might want to let other people see the visuals as well.

## Sharing on your own computer

Nothing could be simpler: just by add `.show()` to the code above, jupyter notebook opens a new window with just this dashboard.

In [ ]:
pn.Column(pn.pane.Markdown("# This is a horizontal barchart"),
          pn.Row(
                 pn.panel("This barchart with 4 categories shows us how we can combine different panes, including vega plots"),
                 pn.pane.Vega(spec)
          )).show()

This starts a new notebook process on your computer and opens a new browser tab with just the dashboard.

## More permanently
In some cases, however, you want to make the dashboard available to a bigger group of people and just want to have it run on some server. In that case we cannot run the notebook interactively and use another solution.

This is a two-step process:

- Instead of `.show()`, call the method `.servable()` on the dashboard that you want to make available. This will not do anything while you are still in the notebook.
- When you have saved the notebook, run `panel serve --show my_notebook.ipynb` on the command line. This will run the complete jupyter notebook in the backend, take the dashboard that can be identified with the `.servable()` method, and create a new interface.

This is the output when I run this command:

![panel serve](https://vda-lab.github.io/visualisation-tutorial/assets/holoviz-servable.png)

This output shows us that the dashboard is running on http://localhost:5006/minimal_vega where other people can access it (obviously replace localhost with the correct IP number).

If you have `.servable()` on more than one dashboard, these will all be included.

# Linking widgets to visuals

With what we’ve learned so far we can create dashboards that contain different graphics, bits of text, etc. But we weren’t able yet to link these things together. That’s what we’ll do here.

## A single widget

As a very first example, we’ll create our simple barchart again, but include a colour picker so that we can change the colour of the bars. Notice that the vega part is exactly what we had before, except for the fact that we also define the color.

In [ ]:
colour_picker = pn.widgets.ColorPicker(value="#4682b4")
colour_picker.width = 50

@pn.depends(colour_picker.param.value)
def plot(value):
    return pn.pane.Vega({
        "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
        "description": "A simple bar chart with embedded data.",
        "height": 100,
        "data": {
          "values": [
            {"a": "A", "b": 28},
            {"a": "B", "b": 55},
            {"a": "C", "b": 43},
            {"a": "D", "b": 91}
          ]
        },
        "mark": "bar",
        "encoding": {
          "x": {"field": "b", "type": "quantitative"},
          "y": {"field": "a", "type": "nominal"},
          "color": {"value": value}
        }
      })
pn.Row(colour_picker, plot)

We create a `pn.Row()` with the `colour_picker` and the plot. How did we make this work?

- The `colour_picker` is a ColorPicker widget. Again, for the complete list of all 30 available widgets, check the panel documentation at https://panel.holoviz.org/user_guide/Widgets.html.
- Instead of just creating the spec as a JSON object, we now create a _function_ (called `plot`) that returns such JSON.
- We need to _decorate_ the plot function so that the function is re-run every time a new colour is chosen. We do this with `@pn.depends(colour_picker.param.value)`. When that `colour_picker.param.value` changes, it will be passed as the first argument to the `plot` function.
- We replaced the actual colour in the specification with `value` which is the name of the first (and only) argument to the function.

From the sci-py lecture notes on decorators:
```
Since functions and classes are objects, they can be passed around. Since they are mutable objects, they can be modified. The act of altering a function or class object after it has been constructed but before it is bound to its name is called decorating.

There are two things hiding behind the name "decorator" — one is the function which does the work of decorating, i.e. performs the real work, and the other one is the expression adhering to the decorator syntax, i.e. an at-symbol and the name of the decorating function.

Function can be decorated by using the decorator syntax for functions:

@decorator             # ②
def function():        # ①
    pass

A function is defined in the standard way. ①
An expression starting with @ placed before the function definition is the decorator ②. The part after @ must be a simple expression, usually this is just the name of a function or class. This part is evaluated first, and after the function defined below is ready, the decorator is called with the newly defined function object as the single argument. The value returned by the decorator is attached to the original name of the function.
```

Note: if you get the error Vega pane does not support objects of type 'dict', make sure you added the schema to your JSON spec: `"$schema": "https://vega.github.io/schema/vega-lite/v4.json"`.

## Multiple widgets

The above is just with one parameter, but we can use more than one. Let’s say we want to also be able to change the width of the plot. We can create a new IntSlider widget, whose value we will use for the width section in the specification.

In [ ]:
colour_picker = pn.widgets.ColorPicker(value="#4682b4")
slider_value = pn.widgets.IntSlider(start = 100, end = 500)

@pn.depends(slider_value.param.value, colour_picker.param.value)
def plot(slider, colour):
    return pn.pane.Vega({
        "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
        "description": "A simple bar chart with embedded data.",
        "height": 100,
        "width": slider,
        "data": {
          "values": [
            {"a": "A", "b": 48},
            {"a": "B", "b": 55},
            {"a": "C", "b": 43},
            {"a": "D", "b": 91}
          ]
        },
        "mark": "bar",
        "encoding": {
          "x": {"field": "b", "type": "quantitative"},
          "y": {"field": "a", "type": "nominal"},
          "color": {"value": colour}
        }
      })
pn.Row(pn.Column(slider_value,colour_picker), plot)

## Actual useful interaction

In the VegaLite tutorial at http:///visualisation-tutorial/vegalite-using-widgets-for-selection.html, we saw how we can define widgets within the specification to select subsets of the data. Let’s try and do the same with HoloViz widgets.

What we had there:
```json
{
  "title": "Making selections",
  "data": {
    "url": "https://raw.githubusercontent.com/vega/vega/master/docs/data/cars.json"
  },
  "selection": {
    "my_selection": {
      "type": "single",
      "fields": ["Origin"],
      "bind": {"input": "select", "options": [null, "Europe", "Japan", "USA"]}
    }
  },
  "mark": "circle",
  "encoding": {
    "x": {"field": "Acceleration", "type": "quantitative"},
    "y": {"field": "Miles_per_Gallon", "type": "quantitative"},
    "color": {
      "condition": {
        "selection": "my_selection",
        "value": "red"
      },
      "value": "lightgrey"
    }
  }
}
```

We can remove the `selection` section in the specification and replace `"selection": "my_selection"` with a test to obtain the following:

In [ ]:
origin_picker = pn.widgets.Select(name="Origin", options=['USA','Europe','Japan'])
@pn.depends(origin_picker.param.value)
def plot2(origin):
    return pn.pane.Vega({
      "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
      "height": 200,
      "title": "Making selections",
      "data": {
        "url": "https://raw.githubusercontent.com/vega/vega/master/docs/data/cars.json"
      },
      "mark": "circle",
      "encoding": {
        "x": {"field": "Acceleration", "type": "quantitative"},
        "y": {"field": "Miles_per_Gallon", "type": "quantitative"},
        "color": {
          "condition": {
            "test": "datum['Origin'] == '" + origin + "'",
            "value": "red"
          },
          "value": "lightgrey"
        }
      }
    })
pn.Row(origin_picker, plot2)

Exercise - Change the last panel so that you can choose what should be put on the y-axis. Possible field names are: “Miles_per_Gallon”, “Displacement”, “Horsepower” or “Weight_in_lbs”. Make sure to use the correct case for these fields! The output should look similar to this:
<img src="https://vda-lab.github.io/visualisation-tutorial/assets/holoviz-widget-exercise.png" width="400"/>

In [ ]:
## Your code for the above exercise goes here

Exercise - Now also add a threshold slider for horsepower (minimum value is 40 and maximum value is 240). Any car with more horsepower than that threshold should be made bigger. The output should look similar to this:

<img src="https://vda-lab.github.io/visualisation-tutorial/assets/holoviz-widget-exercise2.png" width="400"/>

In [ ]:
## Your code for the above exercise goes here

Here’s another example using the Miserables dataset:

In [ ]:
group_picker = pn.widgets.Select(name="Select group", options=[0,1,2,3,4,5,6,7,8], width=50)

@pn.depends(group_picker.param.value)
def plot(gr):
    return pn.pane.Vega({
      "$schema": "https://vega.github.io/schema/vega/v5.json",
      "width": 400,
      "height": 275,
      "padding": 0,
      "autosize": "none",

      "signals": [
        { "name": "cx", "update": "width / 2" },
        { "name": "cy", "update": "height / 2" },
        {
          "description": "State variable for active node dragged status.",
          "name": "dragged", "value": 0,
          "on": [
            {
              "events": "symbol:mouseout[!event.buttons], window:mouseup",
              "update": "0"
            },
            {
              "events": "symbol:mouseover",
              "update": "dragged || 1"
            },
            {
              "events": "[symbol:mousedown, window:mouseup] > window:mousemove!",
              "update": "2", "force": True
            }
          ]
        },
        {
          "description": "Graph node most recently interacted with.",
          "name": "dragged_node", "value": None,
          "on": [
            {
              "events": "symbol:mouseover",
              "update": "dragged === 1 ? item() : dragged_node"
            }
          ]
        },
        {
          "description": "Flag to restart Force simulation upon data changes.",
          "name": "restart", "value": False,
          "on": [
            {"events": {"signal": "dragged"}, "update": "dragged > 1"}
          ]
        }
      ],

      "data": [
        {
          "name": "node-data",
          "url": "https://raw.githubusercontent.com/vega/vega-datasets/master/data/miserables.json",
          "format": {"type": "json", "property": "nodes"}
        },
        {
          "name": "link-data",
          "url": "https://raw.githubusercontent.com/vega/vega-datasets/master/data/miserables.json",
          "format": {"type": "json", "property": "links"}
        }
      ],

      "marks": [
        {
          "name": "nodes",
          "type": "symbol",
          "zindex": 1,

          "from": {"data": "node-data"},
          "on": [
            {
              "trigger": "dragged",
              "modify": "dragged_node",
              "values": "dragged === 1 ? {fx:dragged_node.x, fy:dragged_node.y} : {fx:x(), fy:y()}"
            },
            {
              "trigger": "!dragged",
              "modify": "dragged_node", "values": "{fx: null, fy: null}"
            }
          ],

          "encode": {
            "enter": {
              "fill": [
                { "test": "datum.group == " + str(gr),
                  "value": "red"},
                {"value": "lightgrey"}
              ]
            },
            "update": {
              "size": {"value": 50},
              "cursor": {"value": "pointer"}
            }
          },

          "transform": [
            {
              "type": "force",
              "iterations": 300,
              "velocityDecay": 0.4,
              "restart": {"signal": "restart"},
              "static": False,
              "forces": [
                {"force": "center", "x": {"signal": "cx"}, "y": {"signal": "cy"}},
                {"force": "collide", "radius": 5},
                {"force": "nbody", "strength": -10},
                {"force": "link", "links": "link-data", "distance": 15}
              ]
            }
          ]
        },
        {
          "type": "path",
          "from": {"data": "link-data"},
          "interactive": False,
          "encode": {
            "update": {
              "stroke": {"value": "lightgrey"}
            }
          },
          "transform": [
            {
              "type": "linkpath", "shape": "line",
              "sourceX": "datum.source.x", "sourceY": "datum.source.y",
              "targetX": "datum.target.x", "targetY": "datum.target.y"
            }
          ]
        }
      ]
    })
pn.Column(pn.pane.Markdown("## Highlighting Miserables dataset based on group"),
          pn.Row(group_picker, plot))